In [1]:
from pyspark.sql import SparkSession

import glob

jars = ",".join(glob.glob("/home/jovyan/jars/*.jar"))
     

spark = SparkSession.builder.appName("SparkConsumer") \
                            .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
                            .config("spark.jars", jars) \
                            .config("spark.hadoop.fs.s3a.access.key", "minio123") \
                            .config("spark.hadoop.fs.s3a.secret.key", "minio123") \
                            .config("spark.hadoop.fs.s3a.path.style.access", "true") \
                            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
                            .config('spark.sql.warehouse.dir', "s3a://test/") \
                            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                            .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
                            .config("delta.enable-non-concurrent-writes", "true") \
                            .getOrCreate()

print(spark.sparkContext.getConf().get("spark.jars"))


/home/jovyan/jars/aws-java-sdk-bundle-1.12.262.jar,/home/jovyan/jars/commons-pool2-2.11.1.jar,/home/jovyan/jars/delta-spark_2.12-3.1.0.jar,/home/jovyan/jars/delta-storage-2.4.0.jar,/home/jovyan/jars/hadoop-aws-3.3.4.jar,/home/jovyan/jars/kafka-clients-3.5.0.jar,/home/jovyan/jars/spark-avro_2.12-3.5.0.jar,/home/jovyan/jars/spark-sql-kafka-0-10_2.12-3.5.0.jar,/home/jovyan/jars/spark-token-provider-kafka-0-10_2.12-3.5.0.jar


In [2]:
spark_df = spark.read.format("delta").load("s3a://test/delta/events")

In [3]:
spark_df.count()

6228

In [5]:
spark_df.show()

+-----------+---------+--------------------+--------------------+----------+-------+--------------------+-------------+-------------------+----------+
|   event_id|  user_id|          session_id|                 url|event_type| device|            referrer|    timestamp|         event_time|        dt|
+-----------+---------+--------------------+--------------------+----------+-------+--------------------+-------------+-------------------+----------+
|event_39329|user_1033|e0068e4e-81f4-468...|            main/tag|     click|tablets|http://www.wood.n...|1756654164380|2025-08-31 15:29:24|2025-08-31|
|event_38932|user_1411|e4cdeff9-d511-478...|        app/tag/list|      view| mobile|http://www.stanto...|1756654164389|2025-08-31 15:29:24|2025-08-31|
|event_22301|user_5521|18759d61-fa2b-43a...|                 app|     click|tablets|https://hill.com/...|1756654164395|2025-08-31 15:29:24|2025-08-31|
|event_94059|user_4514|7de50396-0921-4b3...|          wp-content|     click|Desktop|https://ca